In [1]:
import os
import re
import numpy as np
import keras as k
import arena as arn
import arena_bomber as arb
import bot_nn as nn

Using TensorFlow backend.


In [2]:
import ipyparallel as ipp
c = ipp.Client()
dview = c[:]

In [3]:
%%px
import os
import re
import numpy as np

from arena_bomber import ARENA
from bot_nn import BOT

models = [BOT.createmodel(), BOT.createmodel(), BOT.createmodel()]

def game(wids):
    arena = ARENA()
    bots = []
    for wid, model in zip(wids, models):
        model.load_weights("weights_bomber_pool/weights/w{0}.hdf5".format(wid))
        bot = BOT(model=model)
        bots.append(bot)

    arena.registerbot(bots)
    arena.wholegame()
    return arena.winner


[stderr:0] Using TensorFlow backend.
[stderr:1] Using TensorFlow backend.
[stderr:2] Using TensorFlow backend.
[stderr:3] Using TensorFlow backend.
[stderr:4] Using TensorFlow backend.
[stderr:5] Using TensorFlow backend.
[stderr:6] Using TensorFlow backend.
[stderr:7] Using TensorFlow backend.
[stderr:8] Using TensorFlow backend.
[stderr:9] Using TensorFlow backend.
[stderr:10] Using TensorFlow backend.
[stderr:11] Using TensorFlow backend.
[stderr:12] Using TensorFlow backend.
[stderr:13] Using TensorFlow backend.
[stderr:14] Using TensorFlow backend.
[stderr:15] Using TensorFlow backend.
[stderr:16] Using TensorFlow backend.
[stderr:17] Using TensorFlow backend.
[stderr:18] Using TensorFlow backend.
[stderr:19] Using TensorFlow backend.
[stderr:20] Using TensorFlow backend.
[stderr:21] Using TensorFlow backend.
[stderr:22] Using TensorFlow backend.
[stderr:23] Using TensorFlow backend.
[stderr:24] Using TensorFlow backend.
[stderr:25] Using TensorFlow backend.
[stderr:26] Using Tens

In [4]:
@dview.remote(block=True)
def games(n, wids):
    nwids = len(wids)
    sums = np.zeros((nwids,2), int)
    for i in range(n):
        p = sums[:,1]/(sums[:,0] + 1) + np.power(np.log(i+nwids)/(sums[:,0]+1), 0.5) * 1 + 0.000001
        p /= p.sum()
        ids = np.random.choice(np.arange(nwids), 3, False, p)
        winner = game(wids[ids])
        sums[ids, 0] += 1
        if winner == 0:
            sums[ids[0], 1] += 1
        else:
            sums[ids[1], 1] += 1
            sums[ids[2], 1] += 1
    return sums

In [5]:
def f(wids, PERS):
    iter = 0
    PERS = PERS
    nengines = len(dview)
    while True:
        PER = PERS[1] if len(wids) <= 10 else PERS[0]
        n = int(len(wids) * PER / nengines / 3)
        x = games(n, wids)
        y = np.zeros((len(wids),4))
        y[:,0] = wids
        np.add.reduce(x, out=y[:,1:3])
        y[:, 3] = y[:,2]/y[:,1]
        print(y)
        np.savetxt("modeleval{0}.txt".format(iter), y, fmt="%d\t%d\t%d\t%f", delimiter="\t")
        if len(wids) <= 10:
            return y
        winrate = y[:, 3]
        maxwid = np.argmax(winrate)
        thres = winrate[maxwid] - (y[maxwid, 1] * winrate[maxwid] * (1-winrate[maxwid])) ** 0.5 / y[maxwid, 1] * 2.58
        wids = wids[np.where(winrate>thres)]
        iter += 1

In [ ]:
%%time
wids = np.array(
    [
        130000,130500,131000,131500,132000,132500,133000,133500,134000,134500,135000,135500,136000,136500,137000,137500,138000,138500,139000,139500,140000,
        140500,141000,141500,142000,142500,143000,143500,144000,144500,145000,145500,146000,146500,147000,147500,148000,148500,149000,149500,150000,150500,
        151000,151500,152000,152500,153000,153500,154000,154500,155000,155500,156000,156500,157000,157500,158000,158500,159000,159500,160000,160500,161000
    ], int)
y = f(wids, [10000, 60000])